This helper makes sure that the packages that rebuilt successfully with Python 3.10 but fail to install due to their dependencies not yet rebuilt have bugzillas blocked by the appropriate bugzillas of said dependencies.

 1. For every package that has been successfully rebuilt, we see if it has a bugzilla that blocks `PYTHON3.10` and `F35FailsToInstall`.
 2. If so, we parse the initial comment for missing `python3.10dist()` dependencies.
 3. We convert the dependencies to `python3.9dist()` dependencies and find their component.
 4. If the component has a `PYTHON3.10` bugzilla, we mark it as blocking the bugzilla from (1).

In [2]:
from IPython.display import display, Markdown

In [46]:
import bugzilla
bzapi = bugzilla.Bugzilla('bugzilla.redhat.com')
query = bzapi.build_query(product='Fedora')
query['blocks'] = 2016048  # PYTHON3.11
query['limit'] = 20 # Bugzilla page size
query['offset'] = 0
query['status'] = '__open__'
bugz = []
while len(partial := bzapi.query(query)) == 20:
    bugz += partial
    query['offset'] += 20
    print(len(bugz))
bugz += partial

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540


[<Bug #2099191 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d01c60>,
 <Bug #2099189 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d03430>,
 <Bug #2099185 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d03400>,
 <Bug #2099184 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d033d0>,
 <Bug #2099183 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d033a0>,
 <Bug #2099182 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d03370>,
 <Bug #2099181 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d03340>,
 <Bug #2099180 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d03310>,
 <Bug #2099177 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d032e0>,
 <Bug #2099176 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d032b0>,
 <Bug #2099175 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d03280>,
 <Bug #2099174 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d03250>,
 <Bug #2099173 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6d03220>,

In [47]:
F37FailsToInstall = 2045109

In [48]:
python311 = !cat python311.pkgs

In [50]:
len(pure_fti_bugz := [b for b in bugz if b.component in python311 and F37FailsToInstall in b.blocks])
pure_fti_bugz

[<Bug #2098663 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6f67640>,
 <Bug #2098670 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6f65bd0>,
 <Bug #2098681 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6f66200>,
 <Bug #2098697 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6f64430>,
 <Bug #2098698 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1da6f65000>,
 <Bug #2098757 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1db8977a90>,
 <Bug #2098770 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1db8977730>,
 <Bug #2098781 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1db8975e10>,
 <Bug #2098796 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1d9fda2950>,
 <Bug #2098812 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1d9fda3a00>,
 <Bug #2098813 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1d9fda3a30>,
 <Bug #2098814 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1d9fda3a60>,
 <Bug #2098826 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f1d9fda3b80>,

In [52]:
for bug in pure_fti_bugz:
    display(Markdown(f'# {bug.component}\n[rbz#{bug.id}](https://bugzilla.redhat.com/{bug.id})'))
    comments = bug.getcomments()
    initial_comment = comments[0]
    if initial_comment['creator'] != 'mhroncok@redhat.com':
        print('Not an automated FTI bug by Miro')
        continue

    blocked_by = set()
    lines = initial_comment['text'].splitlines()
    for line in lines:
        if line.startswith('  - nothing provides '):
            print(line)
            provide = line.split(' ')[5]
            if provide.startswith('('):
                provide = provide[1:]
            if provide.startswith('python3.11dist('):
                old_provide = provide.replace('python3.11dist(', 'python3.10dist(')
                blocker_components = !repoquery -q --repo=rawhide --source --whatprovides "{old_provide}" | pkgname
                if len(set(blocker_components)) == 1:
                    blocker_component = blocker_components[0]
                    blocker_bugz = [b for b in bugz if b.component == blocker_component]
                    if len(blocker_bugz) > 1:
                        print(f"    = too many bugzillas for {blocker_component}")
                        for bug in blocker_bugz:
                            print(f"        https://bugzilla.redhat.com/{bug.id}")
                    elif blocker_bugz:
                        bid = blocker_bugz[0].id
                        print(f"    = blocked by {blocker_component} https://bugzilla.redhat.com/{bid}")
                        blocked_by.add(bid)
                else:
                    print('Check:', blocker_components)
    if blocked_by:
        add_blockers = bzapi.build_update(depends_on_add=sorted(blocked_by))
        bzapi.update_bugs([bug.id], add_blockers)

# Mayavi
[rbz#2098663](https://bugzilla.redhat.com/2098663)

  - nothing provides python3.11dist(envisage) needed by python3-mayavi-4.7.4-4.fc37.x86_64
    = blocked by python-envisage https://bugzilla.redhat.com/2098901
  - nothing provides python3.11dist(pyface) >= 6.1.1 needed by python3-mayavi-4.7.4-4.fc37.x86_64
    = blocked by python-pyface https://bugzilla.redhat.com/2099067
  - nothing provides python3.11dist(traitsui) >= 7 needed by python3-mayavi-4.7.4-4.fc37.x86_64
    = blocked by python-traitsui https://bugzilla.redhat.com/2099131


# ansible-collections-openstack
[rbz#2098670](https://bugzilla.redhat.com/2098670)

  - nothing provides python3.11dist(openstacksdk) >= 0.36 needed by ansible-collections-openstack-1.7.1-2.938abd0git.fc37.noarch
    = blocked by python-openstacksdk https://bugzilla.redhat.com/2099037


# cekit
[rbz#2098681](https://bugzilla.redhat.com/2098681)

...

  - nothing provides python3.11dist(openstacksdk) needed by sgmanager-2.0.0-12+201801213git+146.861aa67.fc37.noarch
    = blocked by python-openstacksdk https://bugzilla.redhat.com/2099037


# spyder
[rbz#2099175](https://bugzilla.redhat.com/2099175)

  - nothing provides (python3.11dist(python-lsp-server) < 1.5~~ with python3.11dist(python-lsp-server) >= 1.4.1) needed by python3-spyder-5.3.1-3.fc37.noarch
    = blocked by python-lsp-server https://bugzilla.redhat.com/2098993
  - nothing provides (python3.11dist(qdarkstyle) < 3.1~~ with python3.11dist(qdarkstyle) >= 3.0.2) needed by python3-spyder-5.3.1-3.fc37.noarch
    = blocked by python-qdarkstyle https://bugzilla.redhat.com/2099088
  - nothing provides (python3.11dist(spyder-kernels) < 2.4~~ with python3.11dist(spyder-kernels) >= 2.3.1) needed by python3-spyder-5.3.1-3.fc37.noarch
    = blocked by python-spyder-kernels https://bugzilla.redhat.com/2099117
  - nothing provides python3.11dist(cloudpickle) >= 0.5 needed by python3-spyder-5.3.1-3.fc37.noarch
  - nothing provides python3.11dist(cookiecutter) >= 1.6 needed by python3-spyder-5.3.1-3.fc37.noarch
  - nothing provides python3.11dist(python-lsp-black) >= 1.2 needed by python3-spyder-5.3.1-3.fc37.noarch
    = blocked by pyt

# xcat
[rbz#2099189](https://bugzilla.redhat.com/2099189)

  - nothing provides python3.11dist(cchardet) needed by xcat-1.0.4-8.fc37.noarch
    = blocked by python-cchardet https://bugzilla.redhat.com/2021804
